In [1]:
import math
import random
import pandas as pd
import numpy as np
import json
import scipy.special as scs
import ast

In [2]:

def compute_hourly_risk(uni_density,
                        norm_density,
                        skewed_density,
                        people_displacement_within_hour,
                        mdt):
    '''
    An hour can be comprised of 12 sections (5 min interval).
    Each section has their own number of people.
    Each section is multiplied by the densities, sampled twice and computes their risk.

    uni_density: an array of size 'num_hexagons'. Refer to uniform distribution
    norm_density: an array of size 'num_hexagons'. Refer to normal distribution
    skewed_density,: an array of size 'num_hexagons'. Refer to skewed distribution
    people_displacement_within_hour: a dictionary comprised of 12 sections (5 min interval)
                                     and hold the number of people within that section

    mdt: mediam dwell time in that POI
    '''

    # N - Total number of people within the POI in the whole hour
    N = sum(people_displacement_within_hour.values())

    # set the M to be the number of infected individuals in that POI
    T = math.floor(N/3)
    if T != 0:
        dist = [T - i for i in range(T)]
        M = random.choices(range(0, T), dist)
        M = M[0]
    else:
        dist = 0
        M = 0

    uni_risk = 0
    norm_risk = 0
    skewed_risk = 0
    for key, occup in people_displacement_within_hour.items():

        # calculate unifrom densities
        uni_dist_indices_chosen = random.choices(range(len(uni_density)), uni_density, k=occup)
        uni_dist = [0]*len(uni_density)

        for ind in uni_dist_indices_chosen:
            uni_dist[ind] += 1

            # calculate normal densities
        norm_dist_indices_chosen = random.choices(range(len(norm_density)), norm_density, k=occup)
        norm_dist = [0]*len(norm_density)

        for ind in norm_dist_indices_chosen:
            norm_dist[ind] += 1

            # calculate skewed densities
        skewed_dist_indices_chosen = random.choices(range(len(skewed_density)), skewed_density, k=occup)
        skewed_dist = [0]*len(skewed_density)

        for ind in skewed_dist_indices_chosen:
            skewed_dist[ind] += 1

            # calculate uniform risk
        uni_hourly_risk = random.choices(uni_dist,
                                         [1/len(uni_density)]*len(uni_density),
                                         k=2)
        for h in uni_hourly_risk:
            if N-h >= M:
                uni_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))

        # calculate normal risk
        norm_hourly_risk = random.choices(norm_dist,
                                          [1/len(norm_density)]*len(norm_density),
                                          k=2)
        for h in norm_hourly_risk:
            if N-h >= M:
                norm_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))

        # calculate skewed risk
        skewed_hourly_risk = random.choices(skewed_dist,
                                            [1/len(skewed_density)]*len(skewed_density),
                                            k=2)
        for h in skewed_hourly_risk:
            if N-h >= M:
                skewed_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))

    uni_risk_of_infection = 1-(1-(uni_risk / 120.0))**mdt
    uni_risk_of_infection = 0 if uni_risk_of_infection is None else uni_risk_of_infection
    
    normal_risk_of_infection = 1-(1-(norm_risk / 120.0))**mdt
    normal_risk_of_infection = 0 if normal_risk_of_infection is None else normal_risk_of_infection
    
    skewed_risk_of_infection = 1-(1-(skewed_risk / 120.0))**mdt
    skewed_risk_of_infection = 0 if skewed_risk_of_infection is None else skewed_risk_of_infection

    return uni_risk_of_infection, normal_risk_of_infection, skewed_risk_of_infection


def compute_hourly_risks(uni_density,
                         norm_density,
                         skewed_density,
                         people_displacement_within_hours,
                         mdt):
    '''
    uni_density: an array of size 'num_hexagons'. Refer to uniform distribution
    norm_density: an array of size 'num_hexagons'. Refer to normal distribution
    skewed_density,: an array of size 'num_hexagons'. Refer to skewed distribution
    people_displacement_within_hours: an array of dictionaries, each comprises of 12 sections (5 min interval)
                                     that hold the number of people within that section

    mdt: mediam dwell time in that POI
    mdt: mediam dwell time in that POI
    '''
    uni_risks = []
    norm_risks = []
    skewed_risks = []

    for i in range(len(people_displacement_within_hours)):
        uni_risk, norm_risk, skewed_risk = compute_hourly_risk(uni_density,
                                                               norm_density,
                                                               skewed_density,
                                                               people_displacement_within_hours[i],
                                                               mdt)

        uni_risks.append(uni_risk)
        norm_risks.append(norm_risk)
        skewed_risks.append(skewed_risk)

    return uni_risks, norm_risks, skewed_risks


def get_poi_occupancy_within_an_hour(json_string, num_people_in_that_hour):
    '''
    returns a dictionary that specifies of how many people are within a specific time slot of a 5 min increment
    '''
    dwell_time = json.loads(json_string)

    # assigns each person a dwell time based on dwell time distribution
    sum_people = dwell_time['<5'] + dwell_time['5-10'] + dwell_time['11-20'] + dwell_time['21-60'] + dwell_time['61-120'] + dwell_time['121-240'] + dwell_time['>240']

    people_dist_dwell = random.choices([
        2.5,
        7.5,
        15.5,
        40.0
    ],
        [dwell_time['<5'] / sum_people,
         dwell_time['5-10'] / sum_people,
         dwell_time['11-20'] / sum_people,
         (dwell_time['21-60'] + dwell_time['61-120'] + dwell_time['121-240'] + dwell_time['>240'])/sum_people
         ], k=int(float(num_people_in_that_hour))
    )

    # assign a time of arrival for each person
    people_arrival = random.choices([
        0, 5, 10, 15, 20, 25, 30,
        35, 40, 45, 50, 55
    ], k=int(num_people_in_that_hour))

    # record results
    result = {"0-4": 0,
              "5-9": 0,
              "10-14": 0,
              "15-19": 0,
              "20-24": 0,
              "25-29": 0,
              "30-34": 0,
              "35-39": 0,
              "40-44": 0,
              "45-49": 0,
              "50-54": 0,
              "55-59": 0,
              }

    for i in range(int(num_people_in_that_hour)):
        if 0 <= people_arrival[i] < 4 or \
                0 <= people_arrival[i]+people_dist_dwell[i] < 5:
            result["0-4"] +=1
        elif 5 <= people_arrival[i] < 10 or \
                5 <= people_arrival[i]+people_dist_dwell[i] < 10:
            result["5-9"] +=1
        elif 10 <= people_arrival[i] < 15 or \
                10 <= people_arrival[i]+people_dist_dwell[i] < 15:
            result["10-14"] +=1
        elif 15 <= people_arrival[i] < 20 or \
                15 <= people_arrival[i]+people_dist_dwell[i] < 20:
            result["15-19"] +=1
        elif 20 <= people_arrival[i] < 25 or \
                20 <= people_arrival[i]+people_dist_dwell[i] < 25:
            result["20-24"] +=1
        elif 25 <= people_arrival[i] < 30 or \
                25 <= people_arrival[i]+people_dist_dwell[i] < 30:
            result["25-29"] +=1
        elif 30 <= people_arrival[i] < 35 or \
                30 <= people_arrival[i]+people_dist_dwell[i] < 35:
            result["30-34"] +=1
        elif 35 <= people_arrival[i] < 40 or \
                35 <= people_arrival[i]+people_dist_dwell[i] < 40:
            result["35-39"] +=1
        elif 40 <= people_arrival[i] < 45 or \
                40 <= people_arrival[i]+people_dist_dwell[i] < 45:
            result["40-44"] +=1
        elif 45 <= people_arrival[i] < 50 or \
                45 <= people_arrival[i]+people_dist_dwell[i] < 50:
            result["45-49"] +=1
        elif 50 <= people_arrival[i] < 55 or \
                50 <= people_arrival[i]+people_dist_dwell[i] < 55:
            result["50-54"] +=1
        else:
            result["55-59"] +=1

    return result


def get_all_hourly_occupancies(json_string, num_people_in_that_hours):
    '''
    returns an array or dictionaries that specifies of how many people are within a specific time slot of a 5 min increment.
    Each entry refers to one hour of a week
    '''
    results = []

    for i in range(len(num_people_in_that_hours)):
        results.append(get_poi_occupancy_within_an_hour(json_string, num_people_in_that_hours[i]))

    return results


In [3]:
# tmp_file_for_risk_processing.csv was computer from POI-Explorer.ipynb.

tmp = pd.read_csv("tmp_file_for_risk_processing.csv")

# calculate the number of hexagons in each POI
HEX_SIZE = 4 # m^2

tmp['num_hexagons'] = [math.ceil(float(x)/HEX_SIZE) for x in tmp['geodesic_area']]

# set uniform densities
tmp['uniform_densities'] = [[1 / x]*x for x in tmp['num_hexagons']]

# assign random values for each hexagom for each POI
occup = [random.choices(range(0, 30), k=x) for x in tmp['num_hexagons']]
sum_of_occup = [sum(x) for x in occup]

# set skewed densities
tmp['skewed_densities'] = [[float(j / sum_of_occup[i]) for j in occup[i]] for i in range(len(sum_of_occup))]

# set normal densities
std_occup = [np.std(x) for x in occup]
mean_occup = [np.mean(x) for x in occup]
norm_prob_density = [
    [(1 / (std_occup[i] * np.sqrt(2*np.pi))) * np.exp(-0.5*((j-mean_occup[i])/std_occup[i])**2)
     for j in occup[i]]
    for i in range(len(occup))
]

sum_of_occup = [sum(x) for x in norm_prob_density]
tmp['normal_densities'] = [[float(j / sum_of_occup[i]) for j in norm_prob_density[i]] for i in range(len(sum_of_occup))]


<ipython-input-3-edf0f20b47e5>:24: RuntimeWarning: divide by zero encountered in double_scalars
  [(1 / (std_occup[i] * np.sqrt(2*np.pi))) * np.exp(-0.5*((j-mean_occup[i])/std_occup[i])**2)
<ipython-input-3-edf0f20b47e5>:24: RuntimeWarning: invalid value encountered in double_scalars
  [(1 / (std_occup[i] * np.sqrt(2*np.pi))) * np.exp(-0.5*((j-mean_occup[i])/std_occup[i])**2)


In [4]:


tmp['num_people_within_hour'] = [get_all_hourly_occupancies(tmp['bucketed_dwell_times'][x], ast.literal_eval(tmp['true_hourly_visits'][x])) for x in range(len(tmp))]



In [5]:
risks = [compute_hourly_risks(tmp['uniform_densities'][x],
                              tmp['normal_densities'][x],
                              tmp['skewed_densities'][x],
                              tmp['num_people_within_hour'][x],
                              tmp['median_dwell'][x])
         for x in range(len(tmp))]

<ipython-input-2-d92252561dc5>:65: RuntimeWarning: invalid value encountered in double_scalars
  uni_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))
<ipython-input-2-d92252561dc5>:73: RuntimeWarning: invalid value encountered in double_scalars
  norm_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))
<ipython-input-2-d92252561dc5>:81: RuntimeWarning: invalid value encountered in double_scalars
  skewed_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))


In [6]:
risks = np.array(risks)
uniform_risks = risks[:,0]
normal_risks = risks[:,1]
skewed_risks = risks[:,2]



In [7]:
len(skewed_risks[:,167])

37569

In [8]:
for i in range(168):
    tmp['uniform_risks_'+str(i)] = uniform_risks[:,i]
    tmp['normal_risks'+str(i)] = normal_risks[:,i]
    tmp['skewed_risks'+str(i)] = skewed_risks[:,i]

# tmp['uniform_risks'] = uniform_risks
# tmp['normal_risks'] = normal_risks
# tmp['skewed_risks'] = skewed_risks

# tmp.to_csv('GTA_tmp_file_for_risk_processing_with_densities.csv')


In [18]:
# old_risks = pd.read_csv('ca_poi_risks_2021-04-19-one-week.csv')
# # tmp = pd.read_csv('GTA_tmp_file_for_risk_processing_with_densities.csv')
# print(len(tmp))
# tmp = pd.concat([tmp, old_risks[['placekey', 'safegraph_place_id', 'top_category']]], axis=1, join="inner")
# print(len(tmp))


7970
7970


In [9]:
tmp.head(10)

,location_name,true_hourly_visits,tup_key,safegraph_place_id,median_dwell,geodesic_area,latitude,longitude,bucketed_dwell_times,placekey,...,skewed_risks164,uniform_risks_165,normal_risks165,skewed_risks165,uniform_risks_166,normal_risks166,skewed_risks166,uniform_risks_167,normal_risks167,skewed_risks167
0,Shell Oil,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Shell Oil', '35202879')",sg:00112a9a83e34a19a398e278ecafd18f,10.0,1335.233405,43.679416,-79.390149,"{""<5"":2,""5-10"":4,""11-20"":2,""21-60"":1,""61-120"":...",zzw-223@665-zd7-2hq,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,Colborne Lodge,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Colborne Lodge', '35201312')",sg:00c005ad029644f59ea9ca1a050242d0,11.5,744.010137,43.640608,-79.460690,"{""<5"":0,""5-10"":3,""11-20"":2,""21-60"":1,""61-120"":...",zzy-222@665-zbh-d9z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.011914,0.000000,0.000000,0.000000
2,John Bead Outlet,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('John Bead Outlet', '35203532')",sg:0321f3a8c92e4d109c4bb83425b94dbb,21.0,5223.388276,43.734197,-79.286930,"{""<5"":0,""5-10"":1,""11-20"":0,""21-60"":1,""61-120"":...",222-222@665-zt2-j9z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,Taco Bell,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Taco Bell', '35212154')",sg:05aa625a6df14750930cfee4080b0c7a,9.0,237.163221,43.763962,-79.668799,"{""<5"":0,""5-10"":1,""11-20"":1,""21-60"":0,""61-120"":...",222-223@665-zgx-p5f,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,David Leonard Fine Art,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('David Leonard Fine Art', '35202296')",sg:077dcea1a57c439dbedaffbb46d65a1c,83.0,220.884300,43.702072,-79.464787,"{""<5"":0,""5-10"":0,""11-20"":1,""21-60"":2,""61-120"":...",222-224@665-z6p-9s5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.768723,0.617103,0.608103
5,Anytime Fitness,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Anytime Fitness', '35212150')",sg:093da07111ba4a6fbf87953e5b6cf451,19.0,565.875254,43.784431,-79.661567,"{""<5"":0,""5-10"":8,""11-20"":5,""21-60"":4,""61-120"":...",222-222@665-zgr-gp9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
6,Nobleton Custom Woodwork,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Nobleton Custom Woodwork', '35190261')",sg:0c0499a6e1dd43edaab91be9e03e78c3,41.0,12258.350719,43.811377,-79.531200,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":1,""61-120"":...",222-22g@665-z8s-pd9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
7,Raja Fabrics,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Raja Fabrics', '35211696')",sg:0ff2476dbe6449aeb3013813a8592a70,57.0,4004.641704,43.710426,-79.652238,"{""<5"":0,""5-10"":2,""11-20"":2,""21-60"":11,""61-120""...",227-222@665-zhc-xt9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
8,Pinkberry,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Pinkberry', '35201489')",sg:171b5f23289446688a34955d99efd581,29.0,129.723609,43.650371,-79.480943,"{""<5"":0,""5-10"":0,""11-20"":1,""21-60"":1,""61-120"":...",22f-222@665-zbn-5fz,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
9,Riderz Corner,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Riderz Corner', '35200922')",sg:1e69a2a2962f4ff5985d351676414e28,15.0,111.011811,43.658457,-79.424529,"{""<5"":0,""5-10"":1,""11-20"":1,""21-60"":1,""61-120"":...",zzw-223@665-zbq-xh5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [20]:
# tmp = tmp.drop(columns=['Unnamed: 0',
#                         'Unnamed: 0.1',
#                         'tup_key'])
# tmp.head(2)

,location_name,true_hourly_visits,safegraph_place_id,median_dwell,geodesic_area,latitude,longitude,bucketed_dwell_times,num_hexagons,uniform_densities,...,skewed_risks165,uniform_risks_166,normal_risks166,skewed_risks166,uniform_risks_167,normal_risks167,skewed_risks167,placekey,safegraph_place_id,top_category
0,Anytime Fitness,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sg:093da07111ba4a6fbf87953e5b6cf451,19.0,565.875254,43.784431,-79.661567,"{""<5"":0,""5-10"":8,""11-20"":5,""21-60"":4,""61-120"":...",142,"[0.007042253521126761, 0.007042253521126761, 0...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,Specialized Freight Trucking
1,Nobleton Custom Woodwork,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sg:0c0499a6e1dd43edaab91be9e03e78c3,41.0,12258.350719,43.811377,-79.531200,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":1,""61-120"":...",3065,"[0.0003262642740619902, 0.0003262642740619902,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,Restaurants and Other Eating Places


In [10]:
# This file would be used in evaluation.ipynb for further processing tradeoffs
# of different policies and how they affect the risk factor.
# It would also be used in poi_near_me.py to calculate the risk at a poi at a given time

tmp.to_csv('GTA_risks.csv', index=False)
